## Project 5: Contagion risk mapper
**Goal**:
Simulate infection spread using symptom onset and contact logs — and flag crew members who had close contact with a symptomatic individual in the days leading up to their symptom onset.

### 1. Case identification

In [ ]:
import pandas as pd

In [ ]:
contagion_log = pd.read_csv('./data/contagion_log.csv')

In [ ]:
contagion_log.head()

In [ ]:
symptomatic = contagion_log[contagion_log['symptoms'] == 'Yes']

In [ ]:
symptomatic

On 2025-03-27, both:
- Nina Patel
- Emma López

became symptomatic (symptoms == "Yes"). This can be regarded as Day 0 for potential transmission.

### 2. Backward tracing - identifying suspected Source contacts
Purpose:
- Identify the source of the infection
- Understand the epidemiological chain
- Based on: who the symptomatic person was in contact with before symptoms

What happened before Nina Patel and Emma López became symptomatic?
Did one infect the other — or could other crew members have served as potential pre-symptomatic sources or amplifiers within the transmission chain, despite not exhibiting symptoms themselves?

Below we look at Nina Patel and Emma López's contact events in the 3 days before symptom onset, to identify individuals who may have contributed to the spread of infection through close proximity or repeated interactions.

In [ ]:
def contact_tracing(names):
    for name in names:
        filter_name = contagion_log[contagion_log['crew_member']== name]
        filter_name = filter_name[(filter_name['date'] >= '2025-03-24') & (filter_name['date'] <='2025-03-26')]
        print(f"\n{filter_name}")

In [ ]:
contact_tracing(['Nina Patel','Emma López'])

- Nina Patel had contact with Emma López **one day** before **both** became symptomatic.
- **Both** had contact with **Alice Carter 3 days before** symptom onset.
- **Both** had contact with **Jonas Müller** before symptom onset, albeit on different days.

#### Possible interpretations:
1. Possible transmission path from Emma López to Nina Patel (i.e., Emma López infected Nina Patel).
2. Alice Carter was contacted by both symptomatic individulas on the same day, suggesting that she may serve as a **central node** in the contact network.
3. Jonas Müller has separate direct contact with both prior to their symptom onset, placing him in a position of **epidemiological interest**.
   
Although neither has shown symptoms, Alice and Jonas' repeated appearance in exposure chains warrants further investigation to determine whether they acted as asymptomatic spreaders, intermediaries, or were simply coincidental contacts.

In [ ]:
# Tracing Alice Carter and Jonas Müller's contacts before 24th
def contact(names):
    for name in names:
        contact = contagion_log[contagion_log['crew_member'] == name]
        contact = contact[contact['date'] < '2025-03-27']
        print(contact)

In [ ]:
contact2(['Alice Carter','Jonas Müller'])

#### Contact patterns observed
Both Nina and Emma had contact with **Jonas Müller and Alice Carter** between 2025-03-24 and 2025-03-26.

Specifically:
- Nina contacted Jonas on 2025-03-25
- Emma contacted Jonas on 2025-03-26
- Both contacted Alice Carter on 2025-03-24

#### Contact logs for Jonas Müller:
- Jonas contacted Nina (as early as 2025-03-20)Emma (2025-03-21 & 22)
- Jonas was contacted by Alice Carter on 2025-03-24
- Jonas has not reported any symptoms to date

#### Contact logs for Alice Carter:
- Alice was in contact with Ben, Jonas, Leo, and Nina between 2025-03-20 and 2025-03-26
- She was also contacted by both symptomatic individuals on 2025-03-24
- Alice is also not symptomatic, but frequently present in multiple contact chains

### Interpretation & preliminary conclusion
Jonas Müller is the most likely common point of exposure:
- He had earlier contact with both Nina and Emma than any other crew member.
- His interactions fall within plausible incubation windows.
- He has not shown symptoms, suggesting possible asymptomatic transmission.

Alice Carter appears frequently in contact chains and was in touch with both symptomatic crew on the same day (2025-03-24), the only shared exposure date. This positions her as a possible intermediary.

While a definitive transmission source cannot be confirmed without clinical testing, the contact patterns suggest that both Jonas and Alice played key roles in the potential spread. They should be treated as **epidemiologically significant contacts** and monitored accordingly.


### 3. Forward tracing - identifying who might get infected next
Purpose:
- Prevent further spread
- Quarantine or monitor exposed individuals
- Based on: who the symptomatic person contacted before onset

In [ ]:
from datetime import datetime
from datetime import timedelta

In [ ]:
date_new = []
for d in contagion_log['date']:
    date_obj = datetime.strptime(d, "%Y-%m-%d").date()
    date_new.append(date_obj)
contagion_log['date_new'] = date_new
contagion_log.head()

In [ ]:
def contact_chain(names):
    for name in names:
        symptomatic = contagion_log[(contagion_log['crew_member'] == name) & (contagion_log['symptoms'] == 'Yes')]
        date_symp = symptomatic['date_new'].values[0]
        recent_contact = contagion_log[(contagion_log['date_new'] >= date_symp -  timedelta(days=3)) & (contagion_log['date_new'] < date_symp) & (contagion_log['contact_with']==name)]
        print(recent_contact)

In [ ]:
contact_chain(['Nina Patel','Emma López'])

#### Interpretation:
- **Ben O’Hara** had multiple contacts with Nina in the possible infectious window and may be at elevated risk of exposure.
- Emma López was contacted by Nina one day before both became symptomatic — a possible mutual exposure or bidirectional transmission event.

### Crew monitoring summary:

| Crew Member   | Status                          | Justification                                                                 | Recommended Action                       |
|---------------|----------------------------------|--------------------------------------------------------------------------------|------------------------------------------|
| **Nina Patel**    | Symptomatic                      | Confirmed symptomatic on 2025-03-27                                             | 🔴 Isolate immediately                    |
| **Emma López**    | Symptomatic                      | Confirmed symptomatic on 2025-03-27                                             | 🔴 Isolate immediately                    |
| **Ben O’Hara**    | High-risk downstream contact     | Contacted Nina twice during her infectious window                               | 🟠 Active monitoring                      |
| **Alice Carter**  | Upstream contact / network hub   | Contacted by both Nina and Emma on 2025-03-24; central in multiple contact chains | 🟠 Active monitoring                      |
| **Jonas Müller**  | Upstream contact / likely source | Contact with both Nina and Emma prior to onset; no symptoms reported            | 🟠 Active monitoring                      |
| **Leo Zhang**     | Indirect exposure (x2)           | Contacted by Alice and Jonas within exposure window                             | 🟡 Passive monitoring (daily check-ins)   |
